# AirBnB NY Locations Data Case Study

In this final project, you task will be to take the data provided and find evidence to answer the following questions.

1. Which hosts are the busiest and why?
2. How many neighborhood groups are available and which shows up the most?
3. Are private rooms the most popular in manhattan?
4. Which hosts are the busiest and based on their reviews?
5. Which neighorhood group has the highest average price?
6. Which neighborhood group has the highest total price?
7. Which top 5 hosts have the highest total price?
8. Who currently has no (zero) availability with a review count of 100 or more?
9. What host has the highest total of prices and where are they located?
10. When did Danielle from Queens last receive a review?


In [1]:
import numpy as np
import pandas as pd

In [3]:
air_b = pd.read_csv('data/AB_NYC_2019.csv')
air_b.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

#### Quick detour to throw this into a database table for separate SQL analysis

In [4]:
import sqlite3
from sqlite3 import Error
from sqlalchemy import create_engine, text

In [5]:
# open connection to sqllite
conn = sqlite3.connect('data/pythonsqlite.db')

In [6]:
# load dataframes to database
air_b.to_sql('air_b', conn)

48895

In [7]:
conn.close()

#### End of detour section

In [21]:
# How many neighborhood groups are available and which shows up the most?
air_b.groupby(['neighbourhood_group']).count()['id'].to_frame()

# manhattan most available followed closely by brooklyn

,id
neighbourhood_group,
Bronx,1091
Brooklyn,20104
Manhattan,21661
Queens,5666
Staten Island,373


In [90]:
# just noticing "are available"  this was not dealt with earlier
# throwing in that filter to get the same ranking of boroughs of NYC
air_b[(air_b['availability_365'] != 0)].groupby(['neighbourhood_group']).count()['id'].to_frame()

,id
neighbourhood_group,
Bronx,914
Brooklyn,12260
Manhattan,13559
Queens,4298
Staten Island,331


In [91]:
# Are private rooms the most popular in manhattan?
# How many neighborhood groups are available and which shows up the most?
air_b.groupby(['neighbourhood_group' , 'room_type']).count()['id'].to_frame()

# no, entire home/apartments are most popular in Manhattan versus Brooklyn here private rooms most popular

id
neighbourhood_group room_type             
Bronx               Entire home/apt    379
                    Private room       652
                    Shared room         60
Brooklyn            Entire home/apt   9559
                    Private room     10132
                    Shared room        413
Manhattan           Entire home/apt  13199
                    Private room      7982
                    Shared room        480
Queens              Entire home/apt   2096
                    Private room      3372
                    Shared room        198
Staten Island       Entire home/apt    176
                    Private room       188
                    Shared room          9

In [30]:
# Which hosts are the busiest and based on their reviews?
air_b.groupby(['host_name']).count()['id'].sort_values(ascending=False).head(15)

# a descening listing of most busy hosts

host_name
Michael         417
David           403
Sonder (NYC)    327
John            294
Alex            279
Blueground      232
Sarah           227
Daniel          226
Jessica         205
Maria           204
Mike            194
Andrew          190
Anna            187
Laura           182
Chris           182
Name: id, dtype: int64

In [36]:
#Which neighorhood group has the highest average price?
air_b.groupby(['neighbourhood_group'])['price'].mean().round(2)

# unsurprisingly, Manhattan has the highest average price followed by Brooklyn

neighbourhood_group
Bronx             87.50
Brooklyn         124.38
Manhattan        196.88
Queens            99.52
Staten Island    114.81
Name: price, dtype: float64

In [38]:
# Which neighbor hood group has the highest total price?
air_b.groupby(['neighbourhood_group'])['price'].describe().round(2)

# The highest price is suspect.  Using describe to break down the distribution of prices
# it shows how problematic the price is

,count,mean,std,min,25%,50%,75%,max
neighbourhood_group,,,,,,,,
Bronx,1091.0,87.50,106.71,0.0,45.0,65.0,99.0,2500.0
Brooklyn,20104.0,124.38,186.87,0.0,60.0,90.0,150.0,10000.0
Manhattan,21661.0,196.88,291.38,0.0,95.0,150.0,220.0,10000.0
Queens,5666.0,99.52,167.10,10.0,50.0,75.0,110.0,10000.0
Staten Island,373.0,114.81,277.62,13.0,50.0,75.0,110.0,5000.0


In [100]:
''' Putting this into a dataframe for further analysis.  A maximum times standard deviation field
    was calculated clearly showing that the maximum prices are flawed and well outside of the price
    distribution.  Assuming a normal curve, a price that is 3 times the standard deviation from the 
    mean is 99.7% of the data.  Therefore, prices in Brooklyn that are 52.8 times the standard deviation
    conclusively shows flawed data.  All other boroughs contain reviews with flawed pricing data that would
    need to be removed in order to conduct a meaningful study of pricing data.  
'''

df_outliers = air_b.groupby(['neighbourhood_group'])['price'].describe().round(2)

df_outliers['max_x_std'] = ((df_outliers['max'] - df_outliers['mean'])/df_outliers['std']).round(1)

df_outliers

,count,mean,std,min,25%,50%,75%,max,max_x_std
neighbourhood_group,,,,,,,,,
Bronx,1091.0,87.50,106.71,0.0,45.0,65.0,99.0,2500.0,22.6
Brooklyn,20104.0,124.38,186.87,0.0,60.0,90.0,150.0,10000.0,52.8
Manhattan,21661.0,196.88,291.38,0.0,95.0,150.0,220.0,10000.0,33.6
Queens,5666.0,99.52,167.10,10.0,50.0,75.0,110.0,10000.0,59.2
Staten Island,373.0,114.81,277.62,13.0,50.0,75.0,110.0,5000.0,17.6


In [101]:
'''
    A meaningful cleaning would be to remove all pricing data that is greater than 3 standard deviations
'''
# compute the threshold price above the neigbourhood mean by 3 sigma
df_outliers['price_threshold'] = (df_outliers['std'] * 3) - df_outliers['mean']
df_outliers

,count,mean,std,min,25%,50%,75%,max,max_x_std,price_threshold
neighbourhood_group,,,,,,,,,,
Bronx,1091.0,87.50,106.71,0.0,45.0,65.0,99.0,2500.0,22.6,232.63
Brooklyn,20104.0,124.38,186.87,0.0,60.0,90.0,150.0,10000.0,52.8,436.23
Manhattan,21661.0,196.88,291.38,0.0,95.0,150.0,220.0,10000.0,33.6,677.26
Queens,5666.0,99.52,167.10,10.0,50.0,75.0,110.0,10000.0,59.2,401.78
Staten Island,373.0,114.81,277.62,13.0,50.0,75.0,110.0,5000.0,17.6,718.05


In [110]:
# join in threshold value 
air_b = pd.merge(air_b, df_outliers[['price_threshold']], left_on='neighbourhood_group', right_index=True, how='inner')

# convert outliers to null values where above threshold
air_b.loc[air_b.price > air_b.price_threshold] = np.nan

In [114]:
'''
    With the outliers removed, the study above is re-run
'''

# Which neighbor hood group has the highest total price?
air_b.groupby(['neighbourhood_group'])['price'].describe().round(2)

# manhattan has the highest total price

,count,mean,std,min,25%,50%,75%,max
neighbourhood_group,,,,,,,,
Bronx,1049.0,73.31,37.58,0.0,45.0,65.0,90.0,225.0
Brooklyn,19733.0,110.01,69.26,0.0,60.0,90.0,145.0,436.0
Manhattan,21147.0,168.51,105.81,0.0,95.0,146.0,210.0,675.0
Queens,5612.0,91.18,59.04,10.0,50.0,75.0,110.0,400.0
Staten Island,369.0,94.24,75.69,13.0,50.0,75.0,109.0,700.0


In [42]:
#Which top 5 hosts have the highest total price?
# Which neighbor hood group has the highest total price?
air_b.groupby(['host_name'])['price'].max().sort_values(ascending=False).head()

host_name
Kathrine    10000
Erin        10000
Jelena      10000
Amy          9999
Olson        9999
Name: price, dtype: int64

In [115]:
'''
    This study is run again with outliers removed
'''

#Which top 5 hosts have the highest total price?
# Which neighbor hood group has the highest total price?
air_b.groupby(['host_name'])['price'].max().sort_values(ascending=False).head()

host_name
Donna           700.0
Jan             675.0
Red Awning      675.0
ResortShare5    672.0
Juliana         670.0
Name: price, dtype: float64

In [58]:

air_b[(air_b['number_of_reviews'] >=100) & (air_b['availability_365']==0 )].groupby(['host_name'], as_index=False)['number_of_reviews'].sum().sort_values('number_of_reviews', ascending=False)[['host_name','number_of_reviews']].head(20)


,host_name,number_of_reviews
59,Gurpreet Singh,1061
106,Michael,908
38,Deloris,693
23,Brian,501
137,Wanda,480
126,Sofia,432
60,Hayes,396
135,Veronica,351
47,Emily And Joel,335
65,James,320


In [59]:
# What host has the highest total of prices and where are they located?
air_b.groupby(['host_name','neighbourhood_group'])['price'].max().sort_values(ascending=False).head()


host_name  neighbourhood_group
Erin       Brooklyn               10000
Jelena     Manhattan              10000
Kathrine   Queens                 10000
Matt       Manhattan               9999
Amy        Manhattan               9999
Name: price, dtype: int64

In [118]:
'''
    This study is re-run with outliers removed
'''

# What host has the highest total of prices and where are they located?
air_b.groupby(['host_name','neighbourhood_group'])['price'].max().sort_values(ascending=False).head().to_frame()

,,price
host_name,neighbourhood_group,
Donna,Staten Island,700.0
Red Awning,Manhattan,675.0
Jan,Manhattan,675.0
ResortShare5,Manhattan,672.0
Juliana,Manhattan,670.0


In [73]:
# When did Danielle from Queens last receive a review?

# convert field to date type
air_b['last_review'] = pd.to_datetime(air_b['last_review'])

# take max of date field
air_b[(air_b['host_name'] == 'Danielle')].groupby(['host_name'])['last_review'].max()


host_name
Danielle   2019-07-08
Name: last_review, dtype: datetime64[ns]

## Further Questions

1. Which host has the most listings?

In [77]:
air_b.groupby(['host_name'])['id'].count().sort_values(ascending=False).to_frame().head()

,id
host_name,
Michael,417
David,403
Sonder (NYC),327
John,294
Alex,279


2. How many listings have completely open availability?

In [86]:
# select count(*) from air_b where availability_365 = 365

#air_b[(air_b['availability_365'] == 365)]['id'].count()

# SQL means of selection: where availability_365 = 365
# Python df means:        (air_b['availability_365'] == 365)

air_b[(air_b['availability_365'] == 365)]['id'].count()

1295

3. What room_types have the highest review numbers?

In [89]:
air_b.groupby(['room_type'])['number_of_reviews'].sum().sort_values(ascending=False).to_frame()

,number_of_reviews
room_type,
Entire home/apt,580403
Private room,538346
Shared room,19256


# Final Conclusion

In this cell, write your final conclusion for each of the questions asked.

Also, if you uncovered some more details that were not asked above, please describe them here.

-- Add your conclusion --

Prices show strong evidence of data entry errors for price.  Firstly, it's ambiguous if price is per night or per visit.  If per visit with a long minimum_night, perhaps it makes more sense.  Nevertheless, over $3000/month for a furnished room in Astoria makes no sense.  Thus, finding and filtering outliers that make no sense would improve data analysis.